In [2]:
!pip install lm_eval

In [3]:
!nvidia-smi

Thu Oct 24 13:38:19 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:50:00.0 Off |                    0 |
| N/A   37C    P0             87W /  400W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from lm_eval import evaluator
from lm_eval.models.huggingface import HFLM
import torch

2024-10-22 21:20:26.698254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 21:20:26.712576: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 21:20:26.716924: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 21:20:26.729484: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-22 21:20:28.812214: W tensorflow/compiler/tf2

In [4]:
## CONFIG
model_name = "google/gemma-7b-it"


model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

NameError: name 'AutoModelForCausalLM' is not defined

In [5]:
# move model to GPU
device = torch.device("cuda")
model.to(device)


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((3072,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((3072,), eps=1

In [6]:

from lm_eval import evaluator
from lm_eval.models.huggingface import HFLM

model = HFLM(model)
model.model.to(device)


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((3072,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((3072,), eps=1

In [7]:

eval_output = evaluator.simple_evaluate(model=model, tasks=['mmlu'], device='cuda', verbosity='ERROR')

result = eval_output['results']['mmlu']

100%|████████| 311/311 [00:00<00:00, 855.00it/s]
Running loglikelihood requests:  86%|▊| 48261/56

: 

In [11]:
result

{'acc,none': 0.506266913545079,
 'acc_stderr,none': 0.004030016016196707,
 'alias': 'mmlu'}

In [ ]:
result['acc,none']

In [2]:
def check_all_parameters(model):
  return all(p.device.type == 'cuda' for p in model.parameters())


# Evaluate with Linear Intervention

In [1]:
import pandas as pd
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix

from transformers import AutoModelForCausalLM, AutoTokenizer

from lm_eval import evaluator
from lm_eval.models.huggingface import HFLM

device = torch.device("cuda")


from analysis.unsupervised import *
from analysis.supervised import *
from gen_data.get_activations import *

model_name = "google/gemma-7b-it"


2024-10-26 13:20:38.331719: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-26 13:20:38.344886: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-26 13:20:38.348857: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-26 13:20:38.359171: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-26 13:20:40.117684: W tensorflow/compiler/tf2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from gen_data.get_activations import get_res_layers_to_enumerate


def save_model_with_hooks(model, 
                 tokenizer,  
                 probe,
                 layers_to_intervene=None,  # Allow multiple layers
                 intervention_strength=1,
                 tokens="all", add_to="prompt"):
    """
    """
    if layers_to_intervene is None:
        layers_to_intervene = [17]  # Default to layer 17 if no layers specified

    # print out devices 
    # add print statements to see what we are stuck on 
    # try running function while disabling hook 

    def linear_intervene(name):
        def hook(model, input, output):
            if add_to=="prompt":
                if output[0].shape[1] != 1:
                    if tokens=="last":
                        output[0][:, -1, :] += intervention_strength * torch.tensor(probe.coef_[0]).to(device)
                    else:
                        output[0][:, :, :] += intervention_strength * torch.tensor(probe.coef_[0]).to(device)

            else: 
                if tokens=="last":
                    # [1, 8, 3072]
                    output[0][:, -1, :] += intervention_strength * torch.tensor(probe.coef_[0]).to(device)
                    # out = output[0][:, -1, :] +intervention_strength * torch.tensor(probe.coef_[0]).to(device)
                # elif tokens=="prompt":
                    # out = output[0][:, :len_prompt, :] + intervention_strength * torch.tensor(probe.coef_[0]).to(device)
                else:
                    output[0][:, :, :] += intervention_strength * torch.tensor(probe.coef_[0]).to(device)
                    # out = output[0][:, :, :] + intervention_strength * torch.tensor(probe.coef_[0]).to(device) #third colon is token position
            
            return output
        return hook

    layers_to_enum = get_res_layers_to_enumerate(model)
    hooks = []

    # Register hooks for each specified layer
    for layer_index in layers_to_intervene:
        if layer_index < 0 or layer_index >= len(layers_to_enum):
            raise ValueError(f"Layer {layer_index} is out of bounds for the model.")
        hook_handle = model.model.layers[layer_index].register_forward_hook(linear_intervene(layer_index))
        hooks.append(hook_handle)
  


In [11]:
import pickle 
linear_probe_path = "/datasets/Probes/linear_probe.pkl"

with open(linear_probe_path, 'rb') as f:
      saved_log_model = pickle.load(f)

#### Offensive Linear Intervention

In [12]:

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

intervention_strength = 0.25
save_model_with_hooks(model=model, tokenizer=tokenizer, probe=saved_log_model, layers_to_intervene=[16], intervention_strength=intervention_strength, tokens="all", add_to="all")

model.to(device)

hlfm_model = HFLM(model)

check_all_parameters(hlfm_model.model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

True

In [13]:
eval_output = evaluator.simple_evaluate(model=hlfm_model, tasks=['mmlu'], device='cuda', verbosity='ERROR')
eval_output['results']['mmlu']

100%|████████| 311/311 [00:00<00:00, 839.00it/s]
Running loglikelihood requests: 100%|█| 56168/56


{'acc,none': 0.5064805583250249,
 'acc_stderr,none': 0.004030596967044771,
 'alias': 'mmlu'}

In [15]:
result = eval_output['results']['mmlu']

In [16]:
import json
import os

file_path = '/datasets/capabilities/capabilities.json'

# Convert the numeric key to a string
key_string = f"linear {str(intervention_strength)}"

# Check if the file exists
if os.path.exists(file_path):
    with open(file_path, 'r') as json_file:
        existing_data = json.load(json_file)
        
        # If the key already exists, append the result to the list under that key
        if key_string in existing_data:
            existing_data[key_string].append(result)
        else:
            existing_data[key_string] = [result]  # Start a new list if the key doesn't exist
else:
    existing_data = {key_string: [result]}  # Create a new dict with the key and a list containing the result

# Write back to the JSON file
with open(file_path, 'w') as json_file:
    json.dump(existing_data, json_file, indent=4)

#### Defensive linear intervention

In [17]:

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


intervention_strength = -0.25
save_model_with_hooks(model=model, tokenizer=tokenizer, probe=saved_log_model, layers_to_intervene=[16], intervention_strength=intervention_strength, tokens="all", add_to="all")

model.to(device)

hlfm_model = HFLM(model)

check_all_parameters(hlfm_model.model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

True

In [18]:
eval_output = evaluator.simple_evaluate(model=hlfm_model, tasks=['mmlu'], device='cuda', verbosity='ERROR')
eval_output['results']['mmlu']

100%|████████| 311/311 [00:00<00:00, 810.55it/s]
Running loglikelihood requests: 100%|█| 56168/56


{'acc,none': 0.5053411194986469,
 'acc_stderr,none': 0.004027970445943166,
 'alias': 'mmlu'}

In [19]:
result = eval_output['results']['mmlu']

In [20]:
import json
import os

file_path = '/datasets/capabilities/capabilities.json'

# Convert the numeric key to a string
key_string = f"linear {str(intervention_strength)}"

# Check if the file exists
if os.path.exists(file_path):
    with open(file_path, 'r') as json_file:
        existing_data = json.load(json_file)
        
        # If the key already exists, append the result to the list under that key
        if key_string in existing_data:
            existing_data[key_string].append(result)
        else:
            existing_data[key_string] = [result]  # Start a new list if the key doesn't exist
else:
    existing_data = {key_string: [result]}  # Create a new dict with the key and a list containing the result

# Write back to the JSON file
with open(file_path, 'w') as json_file:
    json.dump(existing_data, json_file, indent=4)

# Evaluate with MLP intervention

In [1]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from gen_data.get_activations import get_res_layers_to_enumerate
import torch
import torch.nn as nn
import torch.optim as optim
from gen_data.get_activations import get_res_layers_to_enumerate
from transformers import AutoModelForCausalLM, AutoTokenizer

from lm_eval import evaluator
from lm_eval.models.huggingface import HFLM
from torch.amp import autocast

# def insert_mlp_hook(model, 
#                  mlp,
#                  layers_to_intervene=None,  # Allow multiple layers
#                  target = 0):
#     """
#     """
#     if layers_to_intervene is None:
#         layers_to_intervene = [17]  # Default to layer 17 if no layers specified

#     # print out devices 
#     # add print statements to see what we are stuck on 
#     # try running function while disabling hook 
#     criterion = nn.MSELoss()
#     def linear_intervene(name, target):
#         def hook(model, input, output):
#           if output[0].shape[1] != 1:
#             # print("output shape", output[0].shape) # this sohuld not be 1 at second position
          
#             with torch.enable_grad():
                        
#               # Forward pass
#               feature_vector = output[0][:, :, :].requires_grad_().to(output[0].device)          
#               # print(feature_vector.shape, feature_vector.requires_grad)  # Should print torch.Size([1, 768]) True
              
#               # ## check old predicted class
#               # pred = mlp(feature_vector)

#               # # Apply a sigmoid to convert it into a probability (between 0 and 1)
#               # probability = torch.sigmoid(pred)

#               # # Check which category it belongs to (class 0 or 1)
#               # predicted_class = (probability > 0.5).float()
#               # # print(predicted_class)

#               # print("OLD Predicted class:", predicted_class)
              
#               ### 
              
                      
#               # Define perturbation as a parameter or tensor that requires gradient
#               perturbation = torch.zeros_like(feature_vector, requires_grad=True).to(feature_vector.device)
              
#               printed_once = True

#               for i in range(32):
#                 # Apply perturbation to the feature vector
#                 perturbed_vector = feature_vector + perturbation

#                 # Get output from the MLP
#                 mlp_out = mlp(perturbed_vector)
                
                
#                 target_out = torch.zeros_like(mlp_out)  # Default to zero, or choose an appropriate value

#                 # Target tensor (ensure its shape matches mlp_out's shape)
#                 # target = torch.zeros_like(mlp_out)  # Match the output size
          
#                 if target == 0:
#                   target_out = torch.zeros_like(mlp_out).to(output[0].device) # Match the output size  
#                   if printed_once: 
#                     # print("TARGET", target_out)
#                     printed_once = False
                  
#                 if target == 1:
#                   target_out = torch.ones_like(mlp_out).to(mlp_out.device) # Match the output size 
#                   if printed_once: 
#                     # print("TARGET", target_out) 
#                     printed_once = False

#                 # Calculate the loss
#                 loss = criterion(mlp_out, target_out)

#                 # print(loss, loss.requires_grad)  # Should print a scalar tensor and True

#                 # Backward pass to compute gradients w.r.t the perturbation
#                 loss.backward()

#                 # Access the gradient of the loss w.r.t. the perturbation
#                 grad_input = perturbation.grad

#                 # Define a learning rate
#                 learning_rate = 0.01

#                 # Update the perturbation in the direction of the negative gradient
#                 perturbation = (perturbation - learning_rate * grad_input).clone().detach().requires_grad_().to(feature_vector.device)
                
#               ## check new predicted class
#               # # pred = mlp(perturbation)
#               # pred = mlp(perturbed_vector)

#               # # Apply a sigmoid to convert it into a probability (between 0 and 1)
#               # probability = torch.sigmoid(pred)

#               # # Check which category it belongs to (class 0 or 1)
#               # predicted_class = (probability > 0.5).float()

#               # print("NEW Predicted class:", predicted_class.item())
              
#               ### 
              
              
#               new_out = perturbed_vector, output[1] # concat perturbed vector with the rest of the output

#               # returnn perturbation
#               return new_out
          
#         return hook

#     layers_to_enum = get_res_layers_to_enumerate(model)
#     hooks = []

#     # Register hooks for each specified layer
#     for layer_index in layers_to_intervene:
#         if layer_index < 0 or layer_index >= len(layers_to_enum):
#             raise ValueError(f"Layer {layer_index} is out of bounds for the model.")
#         hook_handle = model.model.layers[layer_index].register_forward_hook(linear_intervene(layer_index, target))
#         hooks.append(hook_handle)
  


2024-10-28 11:36:31.346055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 11:36:31.384453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 11:36:31.388480: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 11:36:31.420898: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-28 11:36:33.750277: W tensorflow/compiler/tf2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from gen_data.get_activations import get_res_layers_to_enumerate
from transformers import AutoModelForCausalLM, AutoTokenizer

from lm_eval import evaluator
from lm_eval.models.huggingface import HFLM
from torch.amp import autocast



def insert_mlp_hook(model, 
                 mlp,
                 layers_to_intervene=None,  # Allow multiple layers
                 offensive = True, 
                 nr_perturbations = 34,
                 learning_rate = 0.005,
                 c = 0):
    """
    """
    if layers_to_intervene is None:
        layers_to_intervene = [17]  # Default to layer 17 if no layers specified

    # print out devices 
    # add print statements to see what we are stuck on 
    # try running function while disabling hook 
    criterion = nn.MSELoss()
    def linear_intervene(name, c, offensive):
        def hook(model, input, output):
          
          with torch.enable_grad():
                      
            # Forward pass
            feature_vector = output[0][:, :, :].requires_grad_().to(output[0].device)          
            # print(feature_vector.shape, feature_vector.requires_grad)  # Should print torch.Size([1, 768]) True
            
            ## check old predicted class
            pred = mlp(feature_vector)
            # print("OLD MLP_out:", pred)
            
            ### 
          
            
                    
            # Define perturbation as a parameter or tensor that requires gradient
            perturbation = torch.zeros_like(feature_vector, requires_grad=True).to(feature_vector.device)
            
            printed_once = False
            
            for i in range(nr_perturbations):
              # Apply perturbation to the feature vector
              perturbed_vector = feature_vector + perturbation

              # # Get output from the MLP
              # if loss == "BCE_new":
              #   mlp_out = sigmoid(mlp(perturbed_vector))
              # else: 
              mlp_out = mlp(perturbed_vector)
              
              mlp_out_value = mlp_out.item()
              
              # target_out = torch.zeros_like(mlp_out)  # Default to zero, or choose an appropriate value
              if offensive:
                  # target = mlp_out_value + c
                  target = max(mlp_out_value+c, c) # if out is very negative, target is c

              else: 
                # defensive
                target = min(mlp_out_value-c, -c) # if out is very positive, target is -c
                    
              
              if not printed_once: 
                # print("target = ", target)
                printed_once = True

              target_out = torch.full_like(mlp_out, target)               

              # Calculate the loss
              loss = criterion(mlp_out, target_out).to(feature_vector.device)

              # print(loss, loss.requires_grad)  # Should print a scalar tensor and True

              # Backward pass to compute gradients w.r.t the perturbation
              loss.backward()

              # Access the gradient of the loss w.r.t. the perturbation
              grad_input = perturbation.grad

              # Define a learning rate
              # learning_rate = 0.01

              
              perturbation = (perturbation - learning_rate * grad_input).clone().detach().requires_grad_().to(output[0].device)

            ## check new predicted class
            # pred = mlp(perturbation)
            pred = mlp(perturbed_vector)
            # print("NEW MLP_out:", pred)
            
            ### 
            
            # Clear unused variables
            del grad_input, mlp_out, loss
            torch.cuda.empty_cache()
                    
            new_out = perturbed_vector, output[1] # concat perturbed vector with the rest of the output

            # returnn perturbation
            return new_out
          
        return hook

    layers_to_enum = get_res_layers_to_enumerate(model)
    hooks = []

    # Register hooks for each specified layer
    for layer_index in layers_to_intervene:
        if layer_index < 0 or layer_index >= len(layers_to_enum):
            raise ValueError(f"Layer {layer_index} is out of bounds for the model.")
        hook_handle = model.model.layers[layer_index].register_forward_hook(linear_intervene(layer_index, c, offensive))
        hooks.append(hook_handle)
  


In [3]:
# def insert_mlp_hook(model, 
#                  mlp,
#                  layers_to_intervene=None,  # Allow multiple layers
#                  target = 0, 
#                  offensive = True, 
#                  nr_perturbations = 34,
#                  learning_rate = 0.005):
#     """
#     """
#     if layers_to_intervene is None:
#         layers_to_intervene = [17]  # Default to layer 17 if no layers specified

#     # print out devices 
#     # add print statements to see what we are stuck on 
#     # try running function while disabling hook 
#     criterion = nn.MSELoss()
#     def linear_intervene(name, c, offensive):
#         def hook(model, input, output):
          
#           with torch.enable_grad(), autocast():
                      
#             # Forward pass
#             feature_vector = output[0][:, :, :].requires_grad_().to(output[0].device)          
#             # print(feature_vector.shape, feature_vector.requires_grad)  # Should print torch.Size([1, 768]) True
            
#             ## check old predicted class
#             pred = mlp(feature_vector)
#             # print("OLD MLP_out:", pred)
            
#             ### 
          
            
                    
#             # Define perturbation as a parameter or tensor that requires gradient
#             perturbation = torch.zeros_like(feature_vector, requires_grad=True).to(feature_vector.device)
            
#             printed_once = False
            
#             for i in range(nr_perturbations):
#               # Apply perturbation to the feature vector
#               perturbed_vector = feature_vector + perturbation

#               # # Get output from the MLP
#               # if loss == "BCE_new":
#               #   mlp_out = sigmoid(mlp(perturbed_vector))
#               # else: 
#               mlp_out = mlp(perturbed_vector)
              
#               mlp_out_value = mlp_out.item()
              
#               # target_out = torch.zeros_like(mlp_out)  # Default to zero, or choose an appropriate value
#               if offensive == True:
#                 if mlp_out_value >= 0:
#                   # If mlp_out_value is positive, add c
#                   target = mlp_out_value + c
#                 else:
#                     # If mlp_out_value is negative, ensure target is still larger than mlp_out_value
#                     target = mlp_out_value + abs(mlp_out_value) + c
                    
#               else: # defensive
#                 if mlp_out_value <= 0:
#                   # If mlp_out_value is positive, add c
#                   target = mlp_out_value - c
#                 else:
#                     # If mlp_out_value is negative, ensure target is still larger than mlp_out_value
#                     target = mlp_out_value - abs(mlp_out_value) - c
                    
              
#               if not printed_once: 
#                 # print("target = ", target)
#                 printed_once = True

#               target_out = torch.full_like(mlp_out, target)               

#               # Calculate the loss
#               loss = criterion(mlp_out, target_out).to(feature_vector.device)

#               # print(loss, loss.requires_grad)  # Should print a scalar tensor and True

#               # Backward pass to compute gradients w.r.t the perturbation
#               loss.backward()

#               # Access the gradient of the loss w.r.t. the perturbation
#               grad_input = perturbation.grad

#               # Define a learning rate
#               # learning_rate = 0.01

              
#               perturbation = (perturbation - learning_rate * grad_input).clone().detach().requires_grad_().to(output[0].device)

#             ## check new predicted class
#             # pred = mlp(perturbation)
#             pred = mlp(perturbed_vector)
#             # print("NEW MLP_out:", pred)
            
#             ### 
            
#             # Clear unused variables
#             del grad_input, mlp_out, loss
#             torch.cuda.empty_cache()
                    
#             new_out = perturbed_vector, output[1] # concat perturbed vector with the rest of the output

#             # returnn perturbation
#             return new_out
          
#         return hook

#     layers_to_enum = get_res_layers_to_enumerate(model)
#     hooks = []

#     # Register hooks for each specified layer
#     for layer_index in layers_to_intervene:
#         if layer_index < 0 or layer_index >= len(layers_to_enum):
#             raise ValueError(f"Layer {layer_index} is out of bounds for the model.")
#         hook_handle = model.model.layers[layer_index].register_forward_hook(linear_intervene(layer_index, target, offensive))
#         hooks.append(hook_handle)
  

In [4]:
probe_path = "/datasets/Probes/mlp_probe_weights.pth"

## CONFIG
model_name = "google/gemma-7b-it"


model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
device = torch.device("cuda")

In [6]:
# del variable
torch.cuda.empty_cache()
model.to(device)

def check_all_parameters(model):
  return all(p.device.type == 'cuda' for p in model.parameters())

hflm_model = HFLM(model)

check_all_parameters(hflm_model.model)

True

In [7]:
from analysis.Probes import SimpleMLP
mlp = SimpleMLP()

# Load the model from the specified probe path
mlp.load_model(probe_path)
mlp.to(device)
check_all_parameters(mlp)

c = 10
insert_mlp_hook(model=model, mlp=mlp, layers_to_intervene=[16], c = c)


Model weights loaded from /data/nathalie_maria_kirch/ERA_Fellowship/experiments/nathaly/mlp_probe_weights.pth!


/data/nathalie_maria_kirch/ERA_Fellowship/analysis/Probes.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(file_path))


In [ ]:
device

device(type='cuda')

In [9]:
def check_hflm_model_devices(hflm_model):
    print("HF Model device check:", check_all_parameters(hflm_model.model))
    for name, param in hflm_model.model.named_parameters():
        # print(f"{name} on {param.device}")
        if param.device.type != 'cuda':
            # print(f"Parameter {name} is not on GPU")
            return False
    return True
        
check_hflm_model_devices(hflm_model)

HF Model device check: True


True

In [10]:
hflm_model.model.to(device)
eval_output = evaluator.simple_evaluate(model=hflm_model, tasks=['mmlu'], device='cuda', verbosity='ERROR')

100%|███████████████| 311/311 [00:00<00:00, 862.47it/s]
Running loglikelihood requests: 100%|█| 56168/56168 [42


In [11]:
result = eval_output['results']['mmlu']

In [12]:
result

{'acc,none': 0.5068366329582681,
 'acc_stderr,none': 0.0040292234599421314,
 'alias': 'mmlu'}

In [13]:
import json
import os

file_path = '/datasets/capabilities/capabilities.json'

# Convert the numeric key to a string
key_string = f"mlp {str(c)}"

# Check if the file exists
if os.path.exists(file_path):
    with open(file_path, 'r') as json_file:
        existing_data = json.load(json_file)
        
        # If the key already exists, append the result to the list under that key
        if key_string in existing_data:
            existing_data[key_string].append(result)
        else:
            existing_data[key_string] = [result]  # Start a new list if the key doesn't exist
else:
    existing_data = {key_string: [result]}  # Create a new dict with the key and a list containing the result

# Write back to the JSON file
with open(file_path, 'w') as json_file:
    json.dump(existing_data, json_file, indent=4)